In [ ]:
%pip install webdriver-manager selenium lzma
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import time
import csv

In [ ]:
print("Installing and running chrome webdriver")
driver = webdriver.Chrome(ChromeDriverManager().install())

In [ ]:
indeedCompanyBaseURL = 'https://www.indeed.com/'

# Change these constants
# Note - do not set the number of reviews to scrape per company over 150
# Otherwise indeed.com will block the scraper, by temporarily blocking your IP Address
# Trust me, I found out the hard way
# Number of Reviews needs to be in multiples of 20
NUMBER_OF_COMPANIES_TO_SCRAPE = 150
NUMBER_OF_REVIEWS_TO_SCRAPE_PER_COMPANY = 100

In [ ]:
companyURLs = {}
print("Beginning to scrape")
companyScraperBaseURL = 'https://www.indeed.com/jobs?q=software+intern&start='
for i in range(0,NUMBER_OF_COMPANIES_TO_SCRAPE,10):
    print("Scraping companies - ",i+10,"/",NUMBER_OF_COMPANIES_TO_SCRAPE)
    url = companyScraperBaseURL+str(i)
    driver.get(url)
    time.sleep(1)
    companies = driver.find_elements(By.CLASS_NAME,'companyOverviewLink')
    for company in companies:
        if company.text not in companyURLs:
            companyURLs[company.text] = (company.get_property('href'))

print("Company Scraping done")

In [ ]:
reviews = {}

for i,company in enumerate(companyURLs):
    url = companyURLs[company]
    print()
    print("({0}/{1}) Scraping company reviews - {2}".format(i+1, len(companyURLs), company))
    for i in range(0,NUMBER_OF_REVIEWS_TO_SCRAPE_PER_COMPANY,20):
        print("Progress - ",i+20,"/",NUMBER_OF_REVIEWS_TO_SCRAPE_PER_COMPANY)
        newUrl = url+'/reviews?&start='+str(i)
        driver.get(newUrl)
        elems = driver.find_elements(By.CLASS_NAME,'eu4oa1w0')
        for elem in elems:
            if elem.tag_name=="span":
                txt = elem.text
                if txt!='':
                    if company not in reviews:
                        reviews[company] = ''
                    reviews[company]+=' '+txt

In [ ]:
import pandas as pd
df = pd.DataFrame()
for company in reviews:
    tempm = {}
    tempm['Company'] = company
    tempm['Reviews'] = reviews[company]
    df = df.append(tempm, ignore_index=True)

df.to_csv('../dataset/out.csv',sep='|', index=False)